<a href="https://colab.research.google.com/github/tejasp2311/AQI_Week1/blob/main/AQI_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline




Load the city_day.csv file into a pandas DataFrame


In [4]:
# Load the dataset
try:
    df = pd.read_csv('city_day.csv')
    print('Dataset loaded successfully')
except FileNotFoundError:
    print('Error: city_day copy.csv not found.')

Dataset loaded successfully!


In [6]:
# Show first 5 rows
print('First 5 rows:')
display(df.head())

print('Last 5 rows:')
display(df.tail())

# Dataset shape
print(f'Dataset shape: {df.shape}')

# Dataset info
print('Dataset info:')
df.info()

# Check missing values
print('Missing values per column:')
display(df.isnull().sum())

First 5 rows:


,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Delhi,2015-01-01,153.3,241.7,182.9,33.0,81.3,38.5,1.87,64.5,83.6,18.93,20.81,8.32,204.5,Severe
1,Mumbai,2015-01-01,70.5,312.7,195.0,42.0,122.5,31.5,7.22,83.8,108.0,2.01,19.41,2.86,60.9,Satisfactory
2,Chennai,2015-01-01,174.1,275.4,56.2,68.8,230.9,28.5,8.56,60.8,43.9,19.07,10.19,9.63,486.5,Severe
3,Kolkata,2015-01-01,477.2,543.9,14.1,76.4,225.9,45.6,2.41,42.1,171.1,9.31,11.65,9.39,174.4,Very Poor
4,Bangalore,2015-01-01,171.6,117.7,123.3,12.4,61.9,49.7,1.26,79.7,164.3,6.04,12.74,9.59,489.7,Good


Last 5 rows:


,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
18260,Delhi,2024-12-31,371.2,61.7,14.6,130.7,159.1,45.9,7.02,76.4,40.0,7.99,5.71,0.36,61.5,Poor
18261,Mumbai,2024-12-31,180.2,259.3,181.9,50.7,135.7,22.4,8.85,12.2,147.3,13.93,7.15,2.08,312.7,Satisfactory
18262,Chennai,2024-12-31,188.6,434.4,68.8,126.1,142.7,47.0,8.30,50.2,128.6,6.98,6.71,2.10,366.1,Good
18263,Kolkata,2024-12-31,343.0,458.6,106.7,108.3,137.8,22.6,2.11,71.4,175.8,4.12,11.68,0.88,397.5,Good
18264,Bangalore,2024-12-31,121.9,100.0,37.7,118.5,224.8,32.6,9.89,16.9,62.6,15.21,4.98,7.06,309.5,Very Poor


Dataset shape: (18265, 16)
Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18265 entries, 0 to 18264
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        18265 non-null  object 
 1   Datetime    18265 non-null  object 
 2   PM2.5       18265 non-null  float64
 3   PM10        18265 non-null  float64
 4   NO          18265 non-null  float64
 5   NO2         18265 non-null  float64
 6   NOx         18265 non-null  float64
 7   NH3         18265 non-null  float64
 8   CO          18265 non-null  float64
 9   SO2         18265 non-null  float64
 10  O3          18265 non-null  float64
 11  Benzene     18265 non-null  float64
 12  Toluene     18265 non-null  float64
 13  Xylene      18265 non-null  float64
 14  AQI         18265 non-null  float64
 15  AQI_Bucket  18265 non-null  object 
dtypes: float64(13), object(3)
memory usage: 2.2+ MB
Missing values per column:


,0
City,0
Datetime,0
PM2.5,0
PM10,0
NO,0
NO2,0
NOx,0
NH3,0
CO,0
SO2,0


In [7]:
# Encode categorical columns (e.g., City)
if 'City' in df.columns:
    le = LabelEncoder()
    df['City'] = le.fit_transform(df['City'].astype(str))

# Feature scaling (StandardScaler for numeric features except AQI)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'AQI' in num_cols:
    num_cols.remove('AQI')
scaler = StandardScaler()
feature_cols = [col for col in num_cols if col in df.columns]
df[feature_cols] = scaler.fit_transform(df[feature_cols])